
# Bert_summarizer


This repo is the generalization of the lecture-summarizer repo. This tool utilizes the HuggingFace Pytorch transformers library to run extractive summarizations. This works by first embedding the sentences, then running a clustering algorithm, finding the sentences that are closest to the cluster's centroids. This library also uses coreference techniques, utilizing the https://github.com/huggingface/neuralcoref library to resolve words in summaries that need more context. The greedyness of the neuralcoref library can be tweaked in the CoreferenceHandler class.

## Install Libraries

In [ ]:
!pip install -q bert-extractive-summarizer
!pip install -q spacy==2.1.3
!pip install -q transformers==2.2.2
!pip install -q neuralcoref

     |████████████████████████████████| 1.9MB 9.7MB/s 
     |████████████████████████████████| 3.2MB 43.6MB/s 
     |████████████████████████████████| 890kB 39.4MB/s 
     |████████████████████████████████| 27.7MB 109kB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 2.1MB 42.7MB/s 
     |████████████████████████████████| 3.2MB 36.4MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.3 which is incompatible.
     |████████████████████████████████| 389kB 8.1MB/s 
     |████████████████████████████████| 133kB 13.7MB/s 
     |████████████████████████████████| 1.2MB 16.6MB/s 
     |████████████████████████████████| 7.3MB 37.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
ERROR: botocore 1.20.27 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 286kB 8.9MB/s 


## Importing Libraires

In [ ]:
from summarizer import Summarizer
from pprint import pprint

## Load Data

In [ ]:
with open("file.txt", 'r') as file:
    data = file.read().replace('\n', '')

In [ ]:
data

'Human language can be inefficient. Some words are vital. Others, expendable.Reread the first sentence of this story. Just two words, “language” and “inefficient,” convey almost the entire meaning of the sentence. The importance of key words underlies a popular new tool for natural language processing (NLP) by computers: the attention mechanism. When coded into a broader NLP algorithm, the attention mechanism homes in on key words rather than treating every word with equal importance. That yields better results in NLP tasks like detecting positive or negative sentiment or predicting which words should come next in a sentence.The attention mechanism’s accuracy often comes at the expense of speed and computing power, however. It runs slowly on general-purpose processors like you might find in consumer-grade computers. So, MIT researchers have designed a combined software-hardware system, dubbed SpAtten, specialized to run the attention mechanism. SpAtten enables more streamlined NLP with

In [ ]:
#data = data.replace("\ufeff", "")

In [ ]:
data[0:1000]

'Human language can be inefficient. Some words are vital. Others, expendable.Reread the first sentence of this story. Just two words, “language” and “inefficient,” convey almost the entire meaning of the sentence. The importance of key words underlies a popular new tool for natural language processing (NLP) by computers: the attention mechanism. When coded into a broader NLP algorithm, the attention mechanism homes in on key words rather than treating every word with equal importance. That yields better results in NLP tasks like detecting positive or negative sentiment or predicting which words should come next in a sentence.The attention mechanism’s accuracy often comes at the expense of speed and computing power, however. It runs slowly on general-purpose processors like you might find in consumer-grade computers. So, MIT researchers have designed a combined software-hardware system, dubbed SpAtten, specialized to run the attention mechanism. SpAtten enables more streamlined NLP with

## Model Summarizer

In [ ]:
model = Summarizer()

In [ ]:
result = model(data, num_sentences=5, min_length=60)

In [ ]:
full = ''.join(result)

## Summarized Output

In [ ]:
pprint(full)

('Just two words, “language” and “inefficient,” convey almost the entire '
 'meaning of the sentence. The importance of key words underlies a popular new '
 'tool for natural language processing (NLP) by computers: the attention '
 'mechanism. It runs slowly on general-purpose processors like you might find '
 'in consumer-grade computers. One challenge he points to is the lack of '
 'specialized hardware to run NLP models with the attention mechanism. '
 'Overall, the software and hardware components of SpAtten combine to '
 'eliminate unnecessary or inefficient data manipulation, focusing only on the '
 'tasks needed to complete the user’s goal. We can improve the battery life '
 'for mobile phone or IoT devices,” says Wang, referring to internet-connected '
 '“things” — televisions, smart speakers, and the like. “')


## Specifying number of sentences for summarization
Number of sentences can be supplied as a ratio or an integer. Examples are provided below.

In [13]:
result = model(data, ratio=0.2)  # Specified with ratio
result = model(data, num_sentences=3)  # Will return 3 sentences 
result

'Just two words, “language” and “inefficient,” convey almost the entire meaning of the sentence. So, MIT researchers have designed a combined software-hardware system, dubbed SpAtten, specialized to run the attention mechanism. Since its introduction in 2015, the attention mechanism has been a boon for NLP. Once the attention mechanism helps pick a sentence’s key words (called tokens), SpAtten prunes away unimportant tokens and eliminates the corresponding computations and data movements.'

## Retrieve Embeddings
You can also retrieve the embeddings of the summarization. Examples are below:

In [14]:
result = model.run_embeddings(data, ratio=0.2)  # Specified with ratio. 
result = model.run_embeddings(data, num_sentences=3)  # Will return (3, N) embedding numpy matrix.
result = model.run_embeddings(data, num_sentences=3, aggregate='mean')  # Will return Mean aggregate over embeddings. 
result

array([-0.32159522, -0.56480616, -0.5650091 , ...,  0.13389969,
        0.63411194,  0.07345836], dtype=float32)

## Simple Example with coreference

In [16]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler

handler = CoreferenceHandler(greedyness=.4)
# How coreference works:
# >>>handler.process('''My sister has a dog. She loves him.''', min_length=2)
# ['My sister has a dog.', 'My sister loves a dog.']

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model1 = Summarizer(sentence_handler=handler)
model1(body)
model1(body2)

## Simple Example with custom model (we always have to set output_hidden_states=True in model config)

In [17]:
from transformers import *

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

from summarizer import Summarizer

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model2 = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
model2(body)
model2(body2)

'Something else you want to summarize with BERT'

## Calculating Elbow
As of bert-extractive-summarizer version 0.7.1, you can also calculate ELBOW to determine the optimal cluster. Below shows a sample example in how to retrieve the list of inertias.

In [11]:
res = model.calculate_elbow(data, k_max=10)
print(res)

[4988.9907, 3563.4455444340074, 3259.4211566014155, 3043.1740047644143, 2861.872693849771, 2722.0706645526134, 2610.160810980346, 2538.248542522099, 2445.4671430932435]


## You can also find the optimal number of sentences with elbow using the following algorithm.

In [21]:
from summarizer import Summarizer

body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model4 = Summarizer()
res = model4.calculate_optimal_k(data, k_max=10)
print(res)